# 사전 훈련된 워드 임베딩(Pre-trained Word Embedding)
- 케라스의 임베딩 층(embedding layer)와 사전 훈련된 워드 임베딩(pre-trained word embedding)을 가져와 사용하는 것을 비교할 것이다

- 자연어 처리를 하려고 할때, 갖고 있는 훈련 데이터의 단어들을 임베딩 층을 구현하여 임베딩 벡터로 학습한다.
    - Keras 에서는 Embedding() 도구를 이용하여 구현한다.
- 위키피디아 등과 같이 방대한 코퍼스를 가지고 Word2Vec, FastText, GloVe를 통해 미리 훈련된 임베딩 벡터를 불러오는 방법도 있다.


### 1. 케라스 임베딩 층 (Keras Embedding Layer)
- Embedding()

#### 1-1) 임베딩 층은 룩업 테이블이다
- 임베딩 층의 입력으로 사용하기 위해서는, 입력 시퀀스의 각 단어들은 모두 정수 인코딩이 되어있어야 한다.
    - 어떤 단어 -> 단어에 부여된 고유한 정수값 -> 임베딩 층 통과 -> 밀집 벡터
    - 임베딩 층 : 입력 정수에 대해 밀집 벡터(dense vector)로 맵핑하고 이는 인공 신경망의 학습 과정에서 가중치가 학습되는 것과 같은 방식으로 훈련된다. ( 가중치 업데이트되는 과정)

<p align='center'><img src='https://wikidocs.net/images/page/33793/lookup_table.PNG'></p>

- 위 그림은 단어'great'를 정수 인코딩하여 해당 임베딩 벡터를 꺼내온다
    - 임베딩 벡터는 모델의 입력이 되고, 역전파 과정에서 단어 great의 임베딩 벡터값이 학습된다.
    - 단어를 정수 인코딩까지만 진행하여 임베딩 층의 입력으로 사용하여 룩업 테이블 결과인, 임베딩 벡터를 리턴해준다

In [ ]:
# 임베딩 층 구현 코드
vocab_size = 20000
output_dim = 128
input_length = 500